In [3]:
import json
import os

import numpy as np
from dotenv import load_dotenv

from apis.openai_api import OpenAIClient
from skills import SKILLS_MAP

load_dotenv()



In [14]:
class ToolsRAG:
    tool_vectors: np.ndarray
    tools: list[str]

    tools_file_path: str
    vectors_file_path: str
    client: OpenAIClient

    def __init__(self, tools_file_path: str = "tools.json", vectors_file_path: str = "tool_vectors.npy"):
        self.tools_file_path = tools_file_path
        self.vectors_file_path = vectors_file_path
        self.client = OpenAIClient()

        self.load_tools_and_vectors()

    def retrieve_tools_from_description(self, description: str, k: int = 5):
        embedding = self.client.embed_text(description)[0]
        similarities = np.dot(self.tool_vectors, embedding)

        top_k_indices = np.argsort(similarities)[-k:][::-1]
        top_k_tools = [self.tools[i] for i in top_k_indices]
        return {t: SKILLS_MAP[t] for t in top_k_tools}

    def load_tools_and_vectors(self) -> None:
        if not os.path.exists(self.tools_file_path):
            raise ValueError(f"Tools file {self.tools_file_path} does not exist.")

        with open(self.tools_file_path, "r") as f:
            descriptions = json.load(f)
        self.tools = [t["name"] for t in descriptions]

        if os.path.exists(self.vectors_file_path):
            self.tool_vectors = np.load(self.vectors_file_path)
            return

        print("Cannot find tool vectors file, generating new vectors...")
        with open(self.tools_file_path, "r") as f:
            tools = json.load(f)
        tools = [f"{t['name']} - {t['description']}" for t in tools]

        self.tool_vectors = self.client.embed_text(tools)
        np.save(self.vectors_file_path, self.tool_vectors)

In [15]:
rag = ToolsRAG()

In [7]:
rag.tools

['add_new_task',
 'gather_routine_information',
 'get_pending_tasks',
 'mark_task_as_done',
 'aggregate_and_email_tasks',
 'shutdown_agent',
 'get_current_time',
 'get_next_24hr_weather_forecast',
 'toggle_voice_mode']

In [16]:
rag.retrieve_tools_from_description("give me an overview of my day")

{'gather_routine_information': <function blueprint_routine.blueprint_skills.gather_routine_information(routine_type: str, day_of_week: str = None) -> str>,
 'aggregate_and_email_tasks': <function skills.email_skills.aggregate_and_email_tasks()>,
 'get_next_24hr_weather_forecast': <function skills.utils.get_next_24hr_weather_forecast() -> str>,
 'get_current_time': <function skills.utils.get_current_time() -> str>,
 'get_pending_tasks': <function skills.notion_skills.get_pending_tasks()>}